In [24]:
from google.colab import drive
import pandas as pd
from tqdm import tqdm

drive.mount('/content/drive')
path = '/content/drive/MyDrive/6000_IMDB_Dataset.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from transformers import AdamW
import torch
review_df = pd.read_csv(path)
feature = review_df['review'].tolist()
label = review_df['sentiment'].tolist()

for i in range(len(label)):
    if label[i] == 'positive':
        label[i] = 1
    else :
        label[i] = 0

train_texts,test_texts, train_labels,test_labels = train_test_split(feature, label, train_size=0.7, shuffle=True)

In [40]:
review_df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,"Petter Mattei's ""Love in the Time of Money"" is...",positive
4,"Probably my all-time favorite movie, a story o...",positive
...,...,...
5995,Something somewhere must have terribly gone wr...,negative
5996,This was the next to last film appearance by J...,negative
5997,I give this movie a 4 cause I'm a die hard fan...,negative
5998,"Are we serious??? I mean wow ... just, wow. I ...",negative


In [42]:
# 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')

# 모델 준비
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
train_encodings ## 이렇게 총 세개의 리턴값을 지님 / input ids는 토큰의 인코딩 / token_type ids 는 nsp를 위해 존재하지만, fine tuning이므로 모두 0 / padding부분에 대해서는 no attention

{'input_ids': tensor([[  101,  2559,  2005,  ...,     0,     0,     0],
        [  101,  2023,  2003,  ...,     0,     0,     0],
        [  101, 18431,  4831,  ...,     0,     0,     0],
        ...,
        [  101,  4406,  1996,  ...,     0,     0,     0],
        [  101,  2096,  2009,  ...,     0,     0,     0],
        [  101,  4567,  2011,  ...,  6429,   999,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [30]:
train_encodings['input_ids'][1]

tensor([  101,  2023,  2003,  2178, 22249,  1998, 14036, 19625,  2143,  1012,
         2508,  5954,  1998, 15467,  2154,  2024,  9788,  1999,  2023,  3185,
         1012,  6795, 23506,  5804,  3544,  2004,  2370,  2379,  1996, 14463,
         1012,  1026,  7987,  1013,  1028,  1026,  7987,  1013,  1028,  1996,
        17363,  1998,  5269,  2024,  2307,  1010,  3272,  1996,  2028,  3496,
         2220,  2006,  2073,  1996,  4281,  2001,  5525,  8275,  1010,  2029,
         2987,  1005,  1056,  2191,  3168,  2000,  2033,  2144,  5019,  2077,
         1998,  2044,  2020,  1999,  1996,  2168,  4292,  1998,  2027,  2020,
         2613,  3295,  7171,  1012,  1045,  1005,  2310,  2657,  2008, 19625,
         2106,  2023,  2006,  3800,  2823,  1012,  1026,  7987,  1013,  1028,
         1026,  7987,  1013,  1028,  1996,  4391,  2005,  2023,  3185,  4025,
         2000,  2022,  3816,  1012,  1045,  2228,  2023,  2003,  1037,  2488,
         2084,  2779, 19625,  3185,  1012,  2200, 14036,  1998, 

In [11]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [43]:
# 훈련 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# 데이터셋 생성
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

epoch_num = 1
model.train()
for epoch in range(epoch_num):
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        print(outputs.logits)
        print(labels)
        print(torch.argmax(outputs.logits, dim=1))
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)
        total += labels.size(0)
        correct += (predicted_labels == labels).sum().item()


accuracy = correct / total
print(f'테스트 정확도: {accuracy}')


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/525 [00:00<?, ?it/s]

tensor([[-0.4843,  0.2016],
        [-0.5413,  0.3139],
        [-0.6480, -0.0895],
        [-0.2257,  0.0779],
        [-0.6491, -0.0397],
        [-0.3469,  0.1913],
        [-0.4686,  0.2648],
        [-0.3464,  0.0487]], grad_fn=<AddmmBackward0>)
tensor([0, 1, 0, 1, 1, 0, 0, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1])


  0%|          | 1/525 [00:03<28:53,  3.31s/it]

tensor([[-0.2142, -0.4169],
        [ 0.0522, -0.2562],
        [-0.0349,  0.1783],
        [-0.1767,  0.2542],
        [-0.1380,  0.0191],
        [-0.1431,  0.0806],
        [-0.2001,  0.1364],
        [ 0.0175,  0.0953]], grad_fn=<AddmmBackward0>)
tensor([1, 0, 1, 1, 0, 1, 1, 1])
tensor([0, 0, 1, 1, 1, 1, 1, 1])


  0%|          | 2/525 [00:06<28:40,  3.29s/it]

...

tensor([[ 0.7141, -1.1034],
        [-1.8173,  1.2384],
        [-0.5839,  0.1885],
        [-2.5169,  2.7301],
        [-1.0883,  0.5101],
        [-2.1433,  2.2060],
        [-1.5537,  1.1328],
        [ 2.7534, -2.7866]], grad_fn=<AddmmBackward0>)
tensor([1, 1, 1, 1, 1, 1, 1, 0])
tensor([0, 1, 1, 1, 1, 1, 1, 0])


100%|█████████▉| 524/525 [28:43<00:03,  3.29s/it]

tensor([[-0.1911, -0.0208],
        [-2.2060,  1.7780],
        [-0.5170,  0.3441],
        [-2.3893,  2.5684],
        [-1.6104,  1.5320],
        [ 0.2809, -0.6234],
        [-0.7211,  0.2889],
        [-1.8638,  1.3864]], grad_fn=<AddmmBackward0>)
tensor([0, 0, 1, 1, 1, 0, 1, 0])
tensor([1, 1, 1, 1, 1, 0, 1, 1])


100%|██████████| 525/525 [28:46<00:00,  3.29s/it]


테스트 정확도: 0.9033333333333333
